# Cartopy tutorial: Appendix for raster to raster

Here are some alternative approaches to exploring the decadal mean surface temperature data using `python-netcdf4` and `xarray`. Originally we were going to include this in the main part of the tutorial, but we felt that it didn't add any real value to the technical narrative of the tutorial.

Rather that purge this contents completely, we decided to archive it here in the appendix, as it may be helpful to any interested tutorial readers.

----

Let's remind ourselves of the source netCDF file containing the pre-processed decadal mean surface temperatures for Oct-Dec 1870s...

In [ ]:
fname = '../../resources/HadCRUT/fogg_mean_surface_temp.nc'

In [ ]:
!ncdump -h $fname

## Using netCDF4 and cartopy

Let's go back to basics and use `netCDF4` to load the appropriate decadal surface temperature data and visulize it using `cartopy`...

In [ ]:
import netCDF4 as nc

In [ ]:
ds = nc.Dataset(fname)

In [ ]:
print(sorted(ds.variables.keys()))

Now we have a handle to the netCDF dataset, we can perform a cursory inspection of its metadata...

In [ ]:
ds.variables['mean_surface_temperature'][:]

In [ ]:
print(ds.variables['decade'][:])

In [ ]:
print(ds.variables['month_number'][:])

Okay, all seem's to be as we expected, so now cherry-pick out the data that we require in order to visualize the decadal mean surface temperature for **October** 1870s...

In [ ]:
data = ds.variables['mean_surface_temperature'][:][0]
lats = ds.variables['latitude'][:]
lons = ds.variables['longitude'][:]

In [ ]:
print('data.shape {}\nlats.shape {}\nlons.shape {}'.format(data.shape, lats.shape, lons.shape))

Before we plot this data, we'll first need to massage the gridded 1-dimensional latitudes and longitudes so that they completely span our 2-dimensional dataset... 

In [ ]:
import numpy as np

xx, yy = np.meshgrid(lons, lats)

Now, let's create a block plot and filled contour plot of the data projected onto a Plate Carree map... 

In [ ]:
%matplotlib notebook

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.figure()

pc = ccrs.PlateCarree()

ax1 = plt.subplot(211, projection=pc)
ax1.pcolormesh(xx, yy, data, transform=pc)
ax1.coastlines()
ax1.set_global()

ax2 = plt.subplot(212, projection=pc)
ax2.contourf(xx, yy, data, transform=pc)
ax2.coastlines()
ax2.set_global()

plt.tight_layout()
plt.show()

## Using xarray and cartopy

Now let's leverage the benefit of `xarray`...

In [ ]:
import xarray as xr

xds = xr.open_dataset(fname)

In [ ]:
xds

In [ ]:
print(xds['decade'])

In [ ]:
print(xds['month_number'])

In [ ]:
print(xds['time'])

In [ ]:
print(xds['time_bnds'])

In [ ]:
plt.figure()

ax = plt.axes(projection=ccrs.PlateCarree())
kwargs = dict(ax=ax, transform=pc, x='longitude', y='latitude', cbar_kwargs=dict(orientation='horizontal'))
xds['mean_surface_temperature'][0].plot.contourf(**kwargs)
ax.coastlines()
ax.set_global()

plt.show()

As from [v0.10.1](http://xarray.pydata.org/en/stable/whats-new.html#id54) of `xarray`, the community can now leverage the benefit of data interoperability between `xarray` and `iris` thanks to the [xarray.DataArray.from_iris](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.from_iris.html) and [xarray.DataArray.to_iris](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.to_iris.html) methods.

In [ ]:
xcube = xds['mean_surface_temperature'].to_iris()

In [ ]:
xcube

In [ ]:
import iris.plot as iplt

plt.figure()

iplt.contourf(xcube[0])
plt.gca().coastlines()

plt.show()